In [19]:
import gym
import random
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.optimizers as ko
from collections import deque
from tqdm import tqdm, notebook  # 학습 과정을 더 깔끔하게 보여주는 library 입니다.

In [26]:
env = gym.make('LunarLander-v2')

print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

# reset은 매 episode가 시작할 때마다 호출해야 합니다.
obs = env.reset()

# random한 action을 뽑아 환경에 적용합니다..
action = env.action_space.sample()
print("Sampled action: {}\n".format(action))
obs, reward, done, info = env.step(action)

# info는 현재의 경우 비어있는 dict지만 debugging과 관련된 정보를 포함할 수 있습니다.
# reward는 scalar 값 입니다.
print("obs : {}\nreward : {}\ndone : {}\ninfo : {}\n".format(obs, reward, done, info))

# 한 episode 에 대한 testing
obs, done, ep_reward = env.reset(), False, 0
print(obs.shape)
print(obs[None].shape)
print(obs.reshape(-1,8).shape)

# 대부분의 gym 환경은 다음과 같은 흐름으로 진행됩니다.
while True: 
    action = env.action_space.sample() # action 선택
    obs, reward, done, info = env.step(action)  # 환경에 action 적용
    ep_reward += reward
    if done:  # episode 종료 여부 체크
        break
        
env.close()  
# Cartpole에서 reward = episode 동안 지속된 step 을 뜻합니다.
print("episode reward : ", ep_reward) 


Observation space: Box(-inf, inf, (8,), float32)
Action space: Discrete(4)
Sampled action: 0

obs : [ 0.00193148  1.4158367   0.0976758   0.09641916 -0.00220708 -0.02189627
  0.          0.        ]
reward : 1.5933365319297934
done : False
info : {}

(8,)
(1, 8)
(1, 8)
episode reward :  -353.27305740410327


In [27]:
units=[64, 64]         # network의 구조. [32, 32]로 설정시 두개의 hidden layer에 32개의 node로 구성된 network가 생성
epsilon=1.0            # epsilon의 초기 값
min_epsilon=.01        # epsilon의 최솟값
epsilon_decay=0.995    # 매 step마다 epsilon이 줄어드는 비율 
train_nums=10000        # train이 진행되는 총 step
gamma=0.97             # discount factor
start_learning = 100

buffer_size=5000        # Replay buffer의 size
batch_size=32          # Repaly buffer로 부터 가져오는 transition minbatch의 크기

target_update_iter=200 # Target network가 update 되는 주기 (step 기준)

In [28]:
# Neural Network Model 
class Model(tf.keras.Model):
    def __init__(self, num_actions, units=[32, 32]):
        super().__init__()
        self.fc1 = kl.Dense(units[0], activation='relu', kernel_initializer='he_uniform')
        self.fc2 = kl.Dense(units[1], activation='relu', kernel_initializer='he_uniform')
        self.logits = kl.Dense(num_actions, name='q_values')

    # forward propagation
    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.logits(x)
        return x

    # return best action that maximize action-value (Q) from network
    # a* = argmax_a' Q(s, a')
    def action_value(self, obs):
        q_values = self.predict(obs)
        best_action = np.argmax(q_values, axis=-1)
        return best_action[0]

In [29]:
class ReplayBuffer:
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.count = 0
        self.buffer = deque(maxlen=buffer_size) 

    # store transition of each step in replay buffer
    def store(self, s, a, r, next_s, d):
        experience = (s, a, r, d, next_s)
        self.buffer.append(experience)
        self.count += 1

    # Sample random minibatch of transtion
    def sample(self, batch_size):
        batch = []
        if self.count < batch_size:
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, batch_size)

        s_batch, a_batch, r_batch, d_batch, s2_batch = map(np.array, list(zip(*batch)))
        return s_batch, a_batch, r_batch, s2_batch, d_batch
    
    def clear(self):
        self.buffer.clear()
        self.count = 0

In [30]:
# test before train
epi_rewards = []
n_episodes = 5
for i in range(n_episodes):
    obs, done, epi_reward = env.reset(), False, 0.0 
    while not done:
        action = 0
        #action = np.random.randint(4)
        next_obs, reward, done, _ = env.step(action)
        epi_reward += reward
        obs = next_obs
    
    print("{} episode reward : {}".format(i, epi_reward))
    epi_rewards.append(epi_reward)

mean_reward = np.mean(epi_rewards)
std_reward = np.std(epi_rewards)

print(f"mean_reward : {mean_reward:.2f} +/- {std_reward:.2f}")

0 episode reward : -137.617808977836
1 episode reward : -150.30338726822242
2 episode reward : -194.83852121823128
3 episode reward : -150.4660593637921
4 episode reward : -136.98497420417925
mean_reward : -154.04 +/- 21.22


In [31]:
replay_buffer = ReplayBuffer(buffer_size)
network = Model(4)
target_network = Model(4)
target_network.set_weights(network.get_weights()) # initialize target network weight 
opt = ko.Adam(learning_rate=.0015, clipvalue=10.0)  # do gradient clip
network.compile(optimizer=opt, loss='mse')

obs = env.reset()
epi_reward = 0.0
epi = 0 # number of episode taken
epsilon=1.0

for t in notebook.tqdm(range(1, train_nums+1), desc='train with DQN'):
    # epsilon update
    if epsilon > min_epsilon:
        epsilon = max(epsilon * epsilon_decay, min_epsilon)

    #######################  step 1  ####################### 
    ####        Select action using episolon-greedy      ### 
    ########################################################   

    # select action that maximize Q value f
    
    best_action = network.action_value(obs[None])  # input the obs to the network model // obs : (4, ) -> obs[None] : (1, 4)
    
    # e-greedy
    if np.random.rand() < epsilon:
        action = env.action_space.sample()
    else:
        action = best_action   # with prob. epsilon, select a random action
    
    #######################  step 2  ####################### 
    #### Take step and store transition to replay buffer ### 
    ########################################################
    
    next_obs, reward, done, _ = env.step(action)    # Excute action in the env to return s'(next state), r, done
    epi_reward += reward
    replay_buffer.store(obs, action, reward, next_obs, done)
    
    #######################  step 3  ####################### 
    ####     Train network (perform gradient descent)    ### 
    ########################################################
    if t > start_learning:
        # target value 계산
        # np.amax -> list 에서 가장 큰 값 반환
        s_batch, a_batch, r_batch, ns_batch, done_batch = replay_buffer.sample(batch_size)
        target_q = r_batch + gamma * np.amax(target_network.predict(ns_batch), axis=1) * (1- done_batch)  
        q_values = network.predict(s_batch) 
        for i, action in enumerate(a_batch):
            q_values[i][action] = target_q[i]

        network.train_on_batch(s_batch, q_values)
    
    #######################  step 3  ####################### 
    ####             Update target network               ### 
    ########################################################
      
    if t % target_update_iter == 0:
        target_network.set_weights(network.get_weights()) # assign the current network parameters to target network
 
    obs = next_obs  # s <- s'
    # if episode ends (done)
    if done:
        epi += 1 # num of episode 
        if epi % 1 == 0:
            print("[Episode {:>5}] epi reward: {:>7.2f}  --eps : {:>4.2f} --steps : {:>5}".format(epi, epi_reward, epsilon, t))
        obs, done, epi_reward = env.reset(), False, 0.0  # Environmnet reset
            

[Episode     1] epi reward: -258.21  --eps : 0.58 --steps :   107
[Episode     2] epi reward: -239.62  --eps : 0.36 --steps :   202
[Episode     3] epi reward: -383.08  --eps : 0.25 --steps :   278
[Episode     4] epi reward:   10.82  --eps : 0.15 --steps :   377
[Episode     5] epi reward: -161.92  --eps : 0.11 --steps :   442
[Episode     6] epi reward: -185.95  --eps : 0.07 --steps :   542
[Episode     7] epi reward:  -31.80  --eps : 0.02 --steps :   742
[Episode     8] epi reward: -259.78  --eps : 0.01 --steps :   838
[Episode     9] epi reward: -265.51  --eps : 0.01 --steps :   971
[Episode    10] epi reward: -125.35  --eps : 0.01 --steps :  1058
[Episode    11] epi reward: -273.70  --eps : 0.01 --steps :  1194
[Episode    12] epi reward: -227.97  --eps : 0.01 --steps :  1449
[Episode    13] epi reward:  -90.29  --eps : 0.01 --steps :  1607
[Episode    14] epi reward: -114.93  --eps : 0.01 --steps :  1797
[Episode    15] epi reward: -122.94  --eps : 0.01 --steps :  2031
[Episode  

In [32]:
# test after train
epi_rewards = []
n_episodes = 5
for i in range(n_episodes):
    obs, done, epi_reward = env.reset(), False, 0.0 
    while not done:
        action = network.action_value(obs[None]) # Using [None] to extend its dimension (4,) -> (1, 4)
        next_obs, reward, done, _ = env.step(action)
        epi_reward += reward
        obs = next_obs
    
    print("{} episode reward : {}".format(i, epi_reward))
    epi_rewards.append(epi_reward)

mean_reward = np.mean(epi_rewards)
std_reward = np.std(epi_rewards)

print(f"mean_reward : {mean_reward:.2f} +/- {std_reward:.2f}")

0 episode reward : -290.3500182318668
1 episode reward : -151.71938543512994
2 episode reward : -73.44159925655296
3 episode reward : -72.34356447080194
4 episode reward : -45.80558311400055
mean_reward : -126.73 +/- 89.15
